In [1]:
# Covid19 Analysis

# Our goal is to built a feature that represent a hospital overload.

In [2]:
# libraries

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from haversine import haversine, Unit
from haversine import haversine_vector

In [ ]:
# reading dataset
# https://opendatasus.saude.gov.br/dataset/bd-srag-2020

# reading dataset

df_2020 = pd.read_csv('/home/pedro/Desktop/code/dataset/INFLUD-21-09-2020.csv',sep=';',encoding = "ISO-8859-1")

# Inputing constraint in the dataset 

# Positive case:

df_2020 = df_2020[df_2020['PCR_SARS2']==1]
print(df_2020.shape)


# Hospitalized people:

df_2020 = df_2020[df_2020['PCR_SARS2']==1][df_2020['HOSPITAL']==1][df_2020['NU_IDADE_N']<=110]
print(df_2020.shape)

# Hospitalized people with age small than 110:

df_2020 = df_2020[df_2020['PCR_SARS2']==1][df_2020['HOSPITAL']==1][df_2020['NU_IDADE_N']<=110][df_2020['EVOLUCAO'] != 3][df_2020['EVOLUCAO'] != 9][df_2020['EVOLUCAO'].notnull()]
print(df_2020.shape)


In [ ]:
# Calculating the number of hospitalization for each hospital separated by epidemiological week

# CNES = hospital code 

In [ ]:
# To build a dataframe

df_2020 = df_2020.groupby(['CO_UNI_NOT', 'SEM_NOT']).size().reset_index(name="Times")

# cheking
print(df_2020)

# size
print(df_2020.shape)

# Nan number
print(df_2020['CO_UNI_NOT'].isna().sum())

In [ ]:
# Times distribution

plt.hist(df_2020['Times'], 100, density=False, facecolor='g', alpha=0.75)
plt.show()

In [ ]:
# max value

print(df_2020['Times'].max())

# hospital code (hospital de campanha)
print(df_2020['CO_UNI_NOT'][df_2020['Times'] == 277])

In [ ]:
# reading dataset
# http://plataforma.saude.gov.br/coronavirus/dados-abertos/

# To analysing the hospitalization by SARS from 2019 to get a proxy of hospital absorption capacity.

df_2019 = pd.read_csv('/home/pedro/Desktop/code/dataset/INFLUD19-16042020.csv',sep=';',encoding = "ISO-8859-1")


In [ ]:
# hospitalization for by hospital


# general data
print(df_2019.shape)

# Selecting hospitalized patients and code from hospital reporting unit.

df_2019 = df_2019['CO_UNI_NOT'][df_2019['HOSPITAL']==1]
print(df_2019.shape)

In [ ]:
# Number of hospitalization in 2019 for each hospital reporting unit.

df_2019 = df_2019.value_counts().rename_axis('CO_UNI_NOT').to_frame('counts')
print(df_2019.shape)

In [ ]:
# max value

print(df_2019['counts'].max())

In [ ]:
# Hospitalization by SARS distribution for 2019

plt.hist(df_2019['counts'], 100, density=False, facecolor='g', alpha=0.75) # .dropna(),
plt.show()

In [ ]:
# Merging both dataset

df = pd.merge(df_2020, df_2019, on='CO_UNI_NOT', how="left")

print(df.shape)

In [ ]:
# Nan number for each variable

print(df['Times'].isna().sum())
print(df['counts'].isna().sum())

In [ ]:
# To check
print(df['Times'].sum())

In [ ]:
# To defininf the Overcrowd feature

df['Overload'] = df['Times']/df['counts']

print(df['Overload'].isna().sum())

In [ ]:
# To building a catalog with reporting health unit, epidemiologic week, and Overcrowded features

df = df[['CO_UNI_NOT','SEM_NOT','Overload']]
print(df.shape)

In [ ]:
# the histogram of the data
plt.hist(df['Overload'], 100, density=False, facecolor='g', alpha=0.75) # .dropna(),
plt.xlabel('Overload')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# To transform in csv dataset

df.to_csv('hospital_overcrowded.csv',index=False)